<a href="https://colab.research.google.com/github/vbonato/cnnTestBench/blob/main/pauNaJaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision





---


---



---
**Dataset WISDM**



---



---



---





In [8]:
# Create time windows

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
import re

# Function to create time windows
def create_time_windows(data, labels, window_size):
    X = []
    y = []

    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])  # Select window of data
        y.append(labels[i + (window_size-1)])  # Label is from the last element of the window

    return np.array(X), np.array(y)

# Updated data loading function
def load_data(file_path):
    features = []
    labels = []

    with open(file_path, 'r') as f:
        for line in f:
            # Use regular expression to match the pattern {{feature_vector}, label}
            match = re.match(r"\{\{([0-9.,-]+)\},\s*(\d+)\}", line.strip())

            if match:
                # Extract feature vector and label
                feature_str = match.group(1)  # The feature string "8.24,-2.11,3.87"
                label = int(match.group(2))  # The label "4"

                # Convert the feature string to a list of floats
                feature_vector = list(map(float, feature_str.split(',')))

                features.append(feature_vector)
                labels.append(label)

    return np.array(features), np.array(labels)

# Load train and test data
train_data_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/WISDM/HAR-Dataset/train.dat'  # Adjust path to your file
test_data_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/WISDM/HAR-Dataset/test.dat'  # Adjust path to your file

# Define the window size
window_size = 200

# Create time windows
X_train, y_train = load_data(train_data_file)
X_train, y_train = create_time_windows(X_train, y_train, window_size)

X_test, y_test = load_data(test_data_file)
X_test, y_test = create_time_windows(X_test, y_test, window_size)

# Print the first feature vector and label
print("First feature vector in X_train:")
print(X_train[0])  # First row (first feature vector)
print("First label in y_train:")
print(y_train[0])  # First label

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)  # For classification (long type for labels)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Print the first feature vector and label
print("First feature vector in X_train:")
print(X_train[0])  # First row (first feature vector)
print("First label in y_train:")
print(y_train[0])  # First label

# Check the shapes of the loaded data
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

First feature vector in X_train:
[[ -0.6946377   12.680544     0.50395286]
 [  5.012288    11.264028     0.95342433]
 [  4.903325    10.882658    -0.08172209]
 [ -0.61291564  18.496431     3.0237172 ]
 [ -1.1849703   12.108489     7.205164  ]
 [  1.3756552   -2.4925237   -6.510526  ]
 [ -0.61291564  10.56939      5.706926  ]
 [ -0.50395286  13.947236     7.0553403 ]
 [ -8.430995    11.413852     5.134871  ]
 [  0.95342433   1.3756552    1.6480621 ]
 [ -8.19945     19.57244      2.7240696 ]
 [  1.4165162    5.7886477    2.982856  ]
 [ -1.879608    -2.982856    -0.29964766]
 [ -6.1291566    6.851035    -8.158588  ]
 [  5.829509    18.0061       8.539958  ]
 [  6.2789803    2.982856     2.9147544 ]
 [ -1.56634      8.308413    -1.4573772 ]
 [  3.5276701   13.593107     9.425281  ]
 [ -2.0294318   -5.706926   -10.18802   ]
 [  2.7649305   10.337844    -9.724928  ]
 [  3.568531    13.6748295    1.5390993 ]
 [ -0.50395286   3.8681788    3.718355  ]
 [ -2.3018389    1.6889231    0.08172209]
 

In [9]:
# Define a model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # CNN layers
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, padding=1)  # input channels = number of features
        #self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, padding=1)  # input channels = number of features

        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        # Fully connected layers
        # Calculate the input size for the fully connected layer based on the output size of conv1
        self.fc1 = nn.Linear(16 * (X_train.shape[1] // 2), 64)  # Flattened size after pooling
        self.fc2 = nn.Linear(64, 6)  # Assuming 6 classes for classification

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Add channel dimension (change shape to [batch_size, channels, seq_length])

        #x = torch.relu(self.conv1(x))  # Apply conv1 and pooling
        #x = self.pool(torch.relu(self.conv2(x)))  # Apply conv2 and pooling
        x = self.pool(torch.relu(self.conv1(x)))  # Apply conv1 and pooling

        x = x.view(-1, 16 * (x.shape[2]))  # Flatten for fully connected layer
        x = torch.relu(self.fc1(x))  # Apply first fully connected layer
        x = self.fc2(x)  # Output layer (no activation since we'll apply softmax in loss)
        return x


# Initialize the model
model = SimpleCNN()

# Loss function (cross-entropy for classification)
criterion = nn.CrossEntropyLoss()

# Optimizer (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# DataLoader for batching
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [10]:
# Training loop
num_epochs = 1  # You can adjust the number of epochs

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass

        # Calculate loss
        loss = criterion(outputs, labels)
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        # Track loss and accuracy
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct_preds / total_preds
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

Epoch 1/1, Loss: 0.1862, Accuracy: 0.9363


In [11]:
# Evaluate the model
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    correct_preds = (predicted == y_test).sum().item()
    accuracy = correct_preds / len(y_test)
    print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.8060


In [12]:
# Save the trained model
torch.save(model.state_dict(), 'simple_cnn_model.pth')

# Load the model (if needed)
model = SimpleCNN()
model.load_state_dict(torch.load('simple_cnn_model.pth'))
model.eval()

SimpleCNN(
  (conv1): Conv1d(3, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=6, bias=True)
)

In [13]:
# --- Post-Training Quantization ---

# 1. Make sure the model is in evaluation mode
model.eval()

# 2. Prepare the model for quantization
# Specify the "backend" for quantization. 'fbgemm' is for x86 CPUs.
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
quantized_model_ptq = torch.quantization.prepare(model, inplace=False) # Use False to keep original model

# 3. Calibrate the model
# We feed the model a few batches of data to observe activation ranges
print("Calibrating the model...")
with torch.no_grad():
    for inputs, _ in train_loader: # Use a few batches from your training data
        quantized_model_ptq(inputs)

print("Calibration done.")

# 4. Convert the model to a quantized version
quantized_model_ptq = torch.quantization.convert(quantized_model_ptq, inplace=True)
print("Model converted to quantized version.")

# Now you can use 'quantized_model_ptq' for inference
# It will be faster and smaller!

Calibrating the model...


/tmp/ipython-input-2572704719.py:9: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  quantized_model_ptq = torch.quantization.prepare(model, inplace=False) # Use False to keep original model


Calibration done.
Model converted to quantized version.


/tmp/ipython-input-2572704719.py:21: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  quantized_model_ptq = torch.quantization.convert(quantized_model_ptq, inplace=True)


In [14]:
pip install onnx onnxruntime

In [17]:

# --- Exports the model structure and weights into a single .onnx file ---

import torch

# Load the saved weights from your training session
model.load_state_dict(torch.load('simple_cnn_model.pth'))
model.eval() # Set the model to evaluation mode

# --- Export to ONNX ---
# Create a dummy input tensor with the correct shape: [batch_size, seq_length, features]
# This shape must match your model's input exactly.
dummy_input = torch.randn(1, window_size, 3)

# Define the output file name
onnx_file_path = "simple_cnn_wisdm.onnx"

print(f"Exporting model to {onnx_file_path}...")

# Export the model
torch.onnx.export(model,               # The model to export
                  dummy_input,         # A sample input
                  onnx_file_path,      # Where to save the model
                  export_params=True,  # Store the trained weights
                  opset_version=12,    # The ONNX version to use
                  input_names=['input'], # The name for the input tensor
                  output_names=['output']) # The name for the output tensor

print("Model has been successfully converted to ONNX format! ✅")

Exporting model to simple_cnn_wisdm.onnx...
Model has been successfully converted to ONNX format! ✅


/tmp/ipython-input-2239025766.py:20: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(model,               # The model to export






---


---



---
**Dataset PAMAP2**



---



---

In [ ]:
#remove columns heartrate and temp of the three IMUs

import pandas as pd

# Define the file path
train_data_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/pamap2.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(train_data_file)

# Remove columns
df = df.drop(df.columns[2], axis=1)
df = df.drop(df.columns[2], axis=1)
df = df.drop(df.columns[11], axis=1)
df = df.drop(df.columns[20], axis=1)


# Save the updated DataFrame to a new CSV file
output_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/pamap2_columns_removed.csv'
df.to_csv(output_file, index=False)

# Print a message to confirm that the file was saved
print(f"Updated CSV saved to: {output_file}")


In [ ]:
# Replace the activity names in the first column with their corresponding IDs

import pandas as pd

# Define the file path for the modified CSV
modified_train_data_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/pamap2_columns_removed.csv'

# Load the modified CSV into a DataFrame without assuming a header
df = pd.read_csv(modified_train_data_file, header=None)

# Print the first few rows to check the content
print("Original DataFrame first few rows:")
print(df.head())

# Create a dictionary to map activity names to activity IDs
activity_map = {
    'lying': 1,
    'sitting': 2,
    'standing': 3,
    'walking': 4,
    'running': 5,
    'cycling': 6,
    'nordic_walking': 7,
    'watching_TV': 9,
    'computer_work': 10,
    'car_driving': 11,
    'ascending_stairs': 12,
    'descending_stairs': 13,
    'vacuum_cleaning': 16,
    'ironing': 17,
    'folding_laundry': 18,
    'house_cleaning': 19,
    'playing_soccer': 20,
    'rope_jumping': 24,
    'other': 0
}

# Replace the activity names in the first column with their corresponding IDs
df.iloc[:, 0] = df.iloc[:, 0].map(activity_map)

# Print the first few rows after the transformation to ensure it's working
print("\nDataFrame after replacing activity names with IDs:")
print(df.head())

# Save the updated DataFrame to a new CSV file
output_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/pamap2_with_activityIDs.csv'
df.to_csv(output_file, index=False, header=False)

# Print a confirmation message
print(f"Updated CSV with activity IDs saved to: {output_file}")


In [ ]:
#Split CSV in train (excluding user 5) and test (only user 5) csv files

import pandas as pd

# Define the file path for the CSV file containing activity IDs
train_data_file_with_ids = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/pamap2_with_activityIDs.csv'

# Load the CSV file into a DataFrame without assuming a header
df = pd.read_csv(train_data_file_with_ids, header=None)

# Print the first few rows to check the content
print("Original DataFrame first few rows:")
print(df.head())

# Filter rows where the user ID is 5 and save them as test_pamap2.csv
test_df = df[df.iloc[:, 1] == 5]
test_df.to_csv('/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/test_pamap2.csv', index=False, header=False)

# Filter rows where the user ID is not 5 (excluding) and save them as train_pamap2.csv
train_df = df[df.iloc[:, 1] != 5]
train_df.to_csv('/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/train_pamap2.csv', index=False, header=False)

# Print a confirmation message
print("Data has been split into train_pamap2.csv (excluding user ID 5) and test_pamap2.csv (user ID 5 only).")


In [ ]:
#Convert .csv to .dat

import pandas as pd

# Define file paths
train_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/train_pamap2.csv'
test_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/test_pamap2.csv'

# Function to convert rows into the required format and save as .dat
def convert_to_dat(input_file, output_file):
    # Load the CSV file without a header
    df = pd.read_csv(input_file, header=None)

    # Open the output .dat file to write
    with open(output_file, 'w') as file:
        for _, row in df.iterrows():
            # Convert row into the required format: {{column 2, column 3, column 4, ...}, column 0}
            row_data = "{{" + ",".join(map(str, row[2:])) + "}," + str(int(row[0])) + "}\n"
            file.write(row_data)

# Convert train_pamap2.csv to train_pamap2.dat
convert_to_dat(train_file, '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/train_pamap2.dat')

# Convert test_pamap2.csv to test_pamap2.dat
convert_to_dat(test_file, '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/test_pamap2.dat')

print("Conversion to .dat files is complete!")


Conversion to .dat files is complete!


In [ ]:
# Create time windows

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
import re

# Function to create time windows
def create_time_windows(data, labels, window_size):
    X = []
    y = []

    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])  # Select window of data
        y.append(labels[i + (window_size-1)])  # Label is from the last element of the window

    return np.array(X), np.array(y)

# Updated data loading function
def load_data(file_path):
    features = []
    labels = []

    with open(file_path, 'r') as f:
        for line in f:
            # Use regular expression to match the pattern {{feature_vector}, label}
            match = re.match(r"\{\{([0-9.,-]+)\},\s*(\d+)\}", line.strip())

            if match:
                # Extract feature vector and label
                feature_str = match.group(1)  # The feature string "8.24,-2.11,3.87"
                label = int(match.group(2))  # The label "4"

                # Convert the feature string to a list of floats
                feature_vector = list(map(float, feature_str.split(',')))

                features.append(feature_vector)
                labels.append(label)

    return np.array(features), np.array(labels)

# Load train and test data
train_data_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/train_pamap2.dat'  # Adjust path to your file
test_data_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/test_pamap2.dat'  # Adjust path to your file

print("File loaded:")

# Define the window size
window_size = 10

# Create time windows
X_train, y_train = load_data(train_data_file)
X_train, y_train = create_time_windows(X_train, y_train, window_size)

X_test, y_test = load_data(test_data_file)
X_test, y_test = create_time_windows(X_test, y_test, window_size)

print("Time window created:")

# Print the first feature vector and label
print("First feature vector in X_train:")
print(X_train[0])  # First row (first feature vector)
print("Second feature vector in X_train:")
print(X_train[1])  # First row (first feature vector)

print("First label in y_train:")
print(y_train[0])  # First label
print("Second label in y_train:")
print(y_train[1])  # First label

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)  # For classification (long type for labels)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# # Print the first feature vector and label
# print("First feature vector in X_train:")
# print(X_train[0])  # First row (first feature vector)
# print("First label in y_train:")
# print(y_train[0])  # First label

# Check the shapes of the loaded data
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
# Define a model

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # CNN layers
        self.conv1 = nn.Conv1d(in_channels=27, out_channels=27, kernel_size=3, padding=1)  # input channels = number of features
        self.conv2 = nn.Conv1d(in_channels=27, out_channels=16, kernel_size=3, padding=1)  # input channels = number of features

        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        # Fully connected layers
        # Calculate the input size for the fully connected layer based on the output size of conv1
        self.fc1 = nn.Linear(16 * (X_train.shape[1] // 2), 64)  # Flattened size after pooling
        self.fc2 = nn.Linear(64, 25)  # Assuming 25 classes for classification

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Add channel dimension (change shape to [batch_size, channels, seq_length])

        x = torch.relu(self.conv1(x))  # Apply conv1 and pooling
        x = self.pool(torch.relu(self.conv2(x)))  # Apply conv2 and pooling
        #x = self.pool(torch.relu(self.conv1(x)))  # Apply conv1 and pooling

        x = x.view(-1, 16 * (x.shape[2]))  # Flatten for fully connected layer
        x = torch.relu(self.fc1(x))  # Apply first fully connected layer
        x = self.fc2(x)  # Output layer (no activation since we'll apply softmax in loss)

        return x


# Initialize the model
model = SimpleCNN()

# Loss function (cross-entropy for classification)
criterion = nn.CrossEntropyLoss()

# Optimizer (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# DataLoader for batching
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# Training loop

num_epochs = 3  # You can adjust the number of epochs

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass

        # Calculate loss
        loss = criterion(outputs, labels)
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        # Track loss and accuracy
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct_preds / total_preds
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

Epoch 1/3, Loss: 0.2979, Accuracy: 0.9041
Epoch 2/3, Loss: 0.1936, Accuracy: 0.9378
Epoch 3/3, Loss: 0.1731, Accuracy: 0.9447


In [ ]:
# Evaluate the model
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    correct_preds = (predicted == y_test).sum().item()
    accuracy = correct_preds / len(y_test)
    print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7133
